In [1]:
def dayOfWeek (year, month, day):
    dayInMonth = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    for i in range(month - 1):
        day += dayInMonth[i]
    day += (year - 2011) * 365
    if (year == 2012) and (month > 2): day += 1
    return ((day - 1)  % 7) + 1

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *
from math import *
from numpy.linalg import norm
%matplotlib inline
import matplotlib.pyplot as plt

train = pd.read_csv('train.csv', delimiter=',')
train.describe()

year = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[0])
year = year.apply(lambda s: int(s))
train.insert(1, 'year', year)

month = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
month = month.apply(lambda s: int(s))
train.insert(2, 'month', month)

day = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[2])
day = day.apply(lambda s: int(s))
dayWeek = np.arange(train.shape[0])
for idx, row in train.iterrows():
    dayWeek[idx] = dayOfWeek(row['year'], row['month'], day[idx])
train.insert(3, 'day', dayWeek)

hour = train['datetime'].apply(lambda s: s.split(' ')[1].split(':')[0])
hour = hour.apply(lambda s: int(s))
train.insert(4, 'hour', hour)

mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
train.insert(5, 'mainHour', mainHour)

season = train['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)
train = train.drop('season', axis = 1)
train.insert(6, 'season', season)

train.head(5)

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

datetime  year  month  day  hour  mainHour  season  holiday  \
0  2011-01-01 00:00:00  2011      1    1     0         0       4        0   
1  2011-01-01 01:00:00  2011      1    1     1         0       4        0   
2  2011-01-01 02:00:00  2011      1    1     2         0       4        0   
3  2011-01-01 03:00:00  2011      1    1     3         0       4        0   
4  2011-01-01 04:00:00  2011      1    1     4         0       4        0   

   workingday  weather  temp   atemp  humidity  windspeed  casual  registered  \
0           0        1  9.84  14.395        81        0.0       3          13   
1           0        1  9.02  13.635        80        0.0       8          32   
2           0        1  9.02  13.635        80        0.0       5          27   
3           0        1  9.84  14.395        75        0.0       3          10   
4           0        1  9.84  14.395        75        0.0       0           1   

   count  
0     16  
1     40  
2     32  
3     13  
4      1

In [3]:
test = pd.read_csv('test.csv', delimiter=',')
test.describe()

year = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[0])
year = year.apply(lambda s: int(s))
test.insert(1, 'year', year)

month = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
month = month.apply(lambda s: int(s))
test.insert(2, 'month', month)

day = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[2])
day = day.apply(lambda s: int(s))
dayWeek = np.arange(test.shape[0])
for idx, row in test.iterrows():
    dayWeek[idx] = dayOfWeek(row['year'], row['month'], day[idx])
test.insert(3, 'day', dayWeek)
    
hour = test['datetime'].apply(lambda s: s.split(' ')[1].split(':')[0])
hour = hour.apply(lambda s: int(s))
test.insert(4, 'hour', hour)

mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
test.insert(5, 'mainHour', mainHour)

season = test['datetime'].apply(lambda s: s.split(' ')[0].split('-')[1])
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)
test = test.drop('season', axis = 1)
test.insert(6, 'season', season)

test.head(5)

datetime  year  month  day  hour  mainHour  season  holiday  \
0  2011-01-20 00:00:00  2011      1    6     0         0       4        0   
1  2011-01-20 01:00:00  2011      1    6     1         0       4        0   
2  2011-01-20 02:00:00  2011      1    6     2         0       4        0   
3  2011-01-20 03:00:00  2011      1    6     3         0       4        0   
4  2011-01-20 04:00:00  2011      1    6     4         0       4        0   

   workingday  weather   temp   atemp  humidity  windspeed  
0           1        1  10.66  11.365        56    26.0027  
1           1        1  10.66  13.635        56     0.0000  
2           1        1  10.66  13.635        56     0.0000  
3           1        1  10.66  12.880        56    11.0014  
4           1        1  10.66  12.880        56    11.0014

In [4]:
X = train.drop(['datetime', 'mainHour', 'casual', 'registered', 'count'], axis = 1)
Xr = train.drop(['datetime', 'casual', 'registered', 'count'], axis = 1)
X_test = test.drop(['datetime', 'mainHour'], axis = 1)
Xr_test = test.drop('datetime', axis = 1)
#X = train[['year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]
#X_test = test[['year', 'month', 'day', 'hour', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']]

In [5]:
maxDepth = 14
nEstimators = 250
subSample = 0.9

y = train['casual']
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)

predictions1 = xgb_model.predict(X)
#-----------------------------------
y = train['registered']
xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(Xr,y)

predictions2 = xgb_model.predict(Xr)
#-----------------------------------
y = train['count']
predictions = predictions1 + predictions2
predictions = np.round(predictions)

print(min(abs(predictions - y)))
print(sum(abs(y - predictions)) / len(predictions))
print(norm(abs(predictions - y), ord = 2))
print(max(abs(predictions - y)))
print(len([elem for elem in (abs(predictions - y) / y) if elem < 0.1]), train.shape[0])

0.0
0.00891052728275
37.7757
22.0
10883 10886


In [6]:
y = train['casual']

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)
predictions1 = xgb_model.predict(X_test)
#--------------------------
y = train['registered']

xgb_model = xgb.XGBRegressor(max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(Xr,y)
predictions2 = xgb_model.predict(Xr_test)
#--------------------------
predictions = predictions1 + predictions2
predictions = np.round(predictions)
predictions = [0 if predictions[i] < 0.01 else predictions[i] for i in range(predictions.shape[0])]

test.insert(len(test.columns), 'count', predictions)
test.head(10)

datetime  year  month  day  hour  mainHour  season  holiday  \
0  2011-01-20 00:00:00  2011      1    6     0         0       4        0   
1  2011-01-20 01:00:00  2011      1    6     1         0       4        0   
2  2011-01-20 02:00:00  2011      1    6     2         0       4        0   
3  2011-01-20 03:00:00  2011      1    6     3         0       4        0   
4  2011-01-20 04:00:00  2011      1    6     4         0       4        0   
5  2011-01-20 05:00:00  2011      1    6     5         0       4        0   
6  2011-01-20 06:00:00  2011      1    6     6         0       4        0   
7  2011-01-20 07:00:00  2011      1    6     7         1       4        0   
8  2011-01-20 08:00:00  2011      1    6     8         1       4        0   
9  2011-01-20 09:00:00  2011      1    6     9         0       4        0   

   workingday  weather   temp   atemp  humidity  windspeed  count  
0           1        1  10.66  11.365        56    26.0027   13.0  
1           1        1  10.66  13.635        56     0.0000    6.0  
2           1        1  10.66  13.635        56     0.0000    4.0  
3           1        1  10.66  12.880        56    11.0014    3.0  
4           1        1  10.66  12.880        56    11.0014    3.0  
5           1        1   9.84  11.365        60    15.0013    6.0  
6           1        1   9.02  10.605        60    15.0013   36.0  
7           1        1   9.02  10.605        55    15.0013   96.0  
8           1        1   9.02  10.605        55    19.0012  227.0  
9           1        2   9.84  11.365        52    15.0013  127.0

In [8]:
newny = pd.DataFrame(test['datetime'])
newny.insert(1, 'count', predictions)
newny.head(20)

newny.to_csv('submission.csv', sep=',', header = True, index = False)